In [1]:
import sys

In [2]:
sys.path.append('/var/www/pipelines/celery_app')

In [3]:
from tasks import get_stock_info, price_range, determine_buy, sort_results

In [4]:
from celery import chain, group, chord

In [5]:
from datetime import datetime

In [6]:
my_stocks = ['FB', 'GOOG', 'IBM']

In [7]:
start = datetime(2016,1,1)
end = datetime.today()

### Working with Chains

In [8]:
my_chain = chain(price_range.s('FB', start, end), determine_buy.s())

In [9]:
my_chain

tasks.price_range('FB', datetime.datetime(2016, 1, 1, 0, 0), datetime.datetime(2016, 8, 22, 9, 47, 10, 633098)) | tasks.determine_buy()

In [10]:
res = my_chain()

In [11]:
res

<AsyncResult: ff86c59a-16f3-4ef8-b12c-6649cd8aea2e>

In [12]:
res.state

'SUCCESS'

In [13]:
res.get()

False

### Working with Groups

In [14]:
my_grp = group(get_stock_info.s(stk, start, end) for stk in my_stocks)

In [15]:
my_grp

[tasks.get_stock_info('FB', datetime.datetime(2016, 1, 1, 0, 0), datetime.datetime(2016, 8, 22, 9, 47, 10, 633098)), tasks.get_stock_info('GOOG', datetime.datetime(2016, 1, 1, 0, 0), datetime.datetime(2016, 8, 22, 9, 47, 10, 633098)), tasks.get_stock_info('IBM', datetime.datetime(2016, 1, 1, 0, 0), datetime.datetime(2016, 8, 22, 9, 47, 10, 633098))]

In [16]:
res = my_grp()

In [17]:
res

<GroupResult: 7a3db195-6340-498b-adb1-f3981dd7667e [1286288f-2622-4dac-83ab-92961be18fe7, 2d6344f5-f03f-47b0-b5e7-399289783e72, b3bf473b-fe6f-4e34-bb17-2dafd80ee35b]>

In [18]:
res.get()

['{"Open min":{"FB":92.830002},"Open max":{"FB":127.519997},"Open mean":{"FB":112.8902501312},"Open median":{"FB":113.8500025},"Low min":{"FB":89.370003},"Low max":{"FB":124.75},"Low mean":{"FB":111.5968122063},"Low median":{"FB":112.959999},"High min":{"FB":95.0},"High max":{"FB":128.330002},"High mean":{"FB":113.9396873375},"High median":{"FB":114.7299995},"Adj Close min":{"FB":94.160004},"Adj Close max":{"FB":125.260002},"Adj Close mean":{"FB":112.8609373937},"Adj Close median":{"FB":113.9300005},"Close min":{"FB":94.160004},"Close max":{"FB":125.260002},"Close mean":{"FB":112.8609373937},"Close median":{"FB":113.9300005}}',
 '{"Open min":{"GOOG":667.849976},"Open max":{"GOOG":785.0},"Open mean":{"GOOG":724.2199360063},"Open median":{"GOOG":722.4649965},"Low min":{"GOOG":663.059998},"Low max":{"GOOG":782.969971},"Low mean":{"GOOG":717.8580696687},"Low median":{"GOOG":716.169983},"Close min":{"GOOG":668.26001},"Close max":{"GOOG":784.849976},"Close mean":{"GOOG":724.18143845},"Close 

### Working with Chords (redis backend only!)

In [19]:
header = [price_range.subtask((stk, start, end)) for stk in my_stocks]

In [23]:
%timeit sort_results(group(header)().get())

1 loop, best of 3: 849 ms per loop


In [24]:
sort_results(group(header)().get())

[{'percent_change': 9.2799999999999994,
  'period_end': datetime.datetime(2016, 8, 22, 9, 47, 10, 633098),
  'period_high': 125.260002,
  'period_low': 94.160004000000001,
  'period_mean': 112.86093739375001,
  'period_start': datetime.datetime(2016, 1, 1, 0, 0),
  'result': 'higher',
  'stock': 'FB',
  'todays_price': 123.34},
 {'percent_change': 11.06,
  'period_end': datetime.datetime(2016, 8, 22, 9, 47, 10, 633098),
  'period_high': 163.529999,
  'period_low': 115.72408999999999,
  'period_mean': 143.70998218125001,
  'period_start': datetime.datetime(2016, 1, 1, 0, 0),
  'result': 'higher',
  'stock': 'IBM',
  'todays_price': 159.61000000000001},
 {'percent_change': 6.54,
  'period_end': datetime.datetime(2016, 8, 22, 9, 47, 10, 633098),
  'period_high': 784.84997599999997,
  'period_low': 668.26000999999997,
  'period_mean': 724.18143844999997,
  'period_start': datetime.datetime(2016, 1, 1, 0, 0),
  'result': 'higher',
  'stock': 'GOOG',
  'todays_price': 771.55999999999995}]

In [25]:
callback = sort_results.subtask()

In [26]:
%timeit chord(header)(callback).get()

1 loop, best of 3: 1.16 s per loop


In [27]:
chord(header)(callback).get()

[{'percent_change': 9.3100000000000005,
  'period_end': datetime.datetime(2016, 8, 22, 9, 47, 10, 633098),
  'period_high': 125.260002,
  'period_low': 94.160004000000001,
  'period_mean': 112.86093739375001,
  'period_start': datetime.datetime(2016, 1, 1, 0, 0),
  'result': 'higher',
  'stock': 'FB',
  'todays_price': 123.37},
 {'percent_change': 11.06,
  'period_end': datetime.datetime(2016, 8, 22, 9, 47, 10, 633098),
  'period_high': 163.529999,
  'period_low': 115.72408999999999,
  'period_mean': 143.70998218125001,
  'period_start': datetime.datetime(2016, 1, 1, 0, 0),
  'result': 'higher',
  'stock': 'IBM',
  'todays_price': 159.61000000000001},
 {'percent_change': 6.5099999999999998,
  'period_end': datetime.datetime(2016, 8, 22, 9, 47, 10, 633098),
  'period_high': 784.84997599999997,
  'period_low': 668.26000999999997,
  'period_mean': 724.18143844999997,
  'period_start': datetime.datetime(2016, 1, 1, 0, 0),
  'result': 'higher',
  'stock': 'GOOG',
  'todays_price': 771.30999